In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data Loading 

In [2]:
sales_df = pd.read_csv('../../datasets/predict-sales/sales_train.csv')
items_df = pd.read_csv('../../datasets/predict-sales/items.csv')
test_df = pd.read_csv('../../datasets/predict-sales/test.csv')

In [3]:
sales_df.drop(labels=['date'],inplace=True,axis=1)
sales_df = sales_df.reset_index()
items_df.drop(labels=['item_name'],inplace=True,axis=1)

In [4]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
index             int64
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(4)
memory usage: 134.4 MB


In [5]:
sales_df.head()

,index,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,0,59,22154,999.00,1.0
1,1,0,25,2552,899.00,1.0
2,2,0,25,2552,899.00,-1.0
3,3,0,25,2554,1709.05,1.0
4,4,0,25,2555,1099.00,1.0


# Visualizing Data 

Let's visualize some data from the test set:

In [ ]:
def drawTimeSerie(df, sample, n):
    for i in range(n):
        shop_id = sample.iloc[i]['shop_id']
        item_id = sample.iloc[i]['item_id']
        id_df = df[(df['shop_id'] == shop_id) & (df['item_id'] == item_id)]
        id_df_grouped = id_df[['date_block_num','item_cnt_day']].groupby('date_block_num').sum().reset_index()
        plt.figure(figsize=[10,n*5])
        plt.subplot(n,1,i+1)
        plt.plot(id_df_grouped['date_block_num'], id_df_grouped['item_cnt_day'],'*-',)
        plt.title(str(shop_id) + '-' + str(item_id))

In [ ]:
n_samples = 10
sample = test_df.sample(n=n_samples, axis=0)

In [ ]:
drawTimeSerie(sales_df, sample, n_samples)

# Structure of Data

In [6]:
import featuretools as ft

Let's evaluate the FT process 

## Pre-proccesing

In [7]:
month_id = 15

In [8]:
sales_month_df = sales_df[sales_df['date_block_num'] == month_id]

In [9]:
sales_month_df['ID_pair'] = sales_month_df[['shop_id','item_id']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

In [10]:
sales_month_df = sales_month_df.merge(items_df)

In [11]:
sales_month_df.head()

,index,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID_pair,item_category_id
0,1549474,15,54,18394,299.0,1.0,54-18394,57
1,1549475,15,54,18394,299.0,1.0,54-18394,57
2,1549482,15,54,18394,299.0,1.0,54-18394,57
3,1562915,15,57,18394,299.0,1.0,57-18394,57
4,1564393,15,24,18394,299.0,1.0,24-18394,57


In [12]:
res_cat_sales = {}

In [13]:
def getResidualCategorySales(x):
    i = str(x[0]) + '-' + str(x[1])
    try:
        return res_cat_sales[i]
    except:
        res_cat_info = sales_month_df[(sales_month_df['shop_id'] == x[0]) &
                             (sales_month_df['item_id'] != x[1]) &
                             (sales_month_df['item_category_id'] != x[2])][['item_price','item_cnt_day']]
        count = res_cat_info.count()[0]
        mean = res_cat_info.mean()
        std = res_cat_info.std()
        maxi = res_cat_info.max()
        mini = res_cat_info.min()        
        skew = res_cat_info.skew()
        if (count > 0):
            result = (count,mean[0],mean[1],std[0],std[1],maxi[0],maxi[1],mini[0],mini[1],skew[0],skew[1])
        else:
            result = (0,-1,-1,-1,-1,-1,-1)
        res_cat_sales[i] = result
        return result

In [15]:
# Testing function
#getResidualCategorySales([54,18394,57])

In [16]:
sales_month_df['res_cat_sales'] = sales_month_df[['shop_id','item_id','item_category_id']].apply(getResidualCategorySales, axis=1)

In [17]:
sales_month_df['res_cat_count'] = sales_month_df['res_cat_sales'].apply(lambda x: x[0])
sales_month_df['res_cat_mean_item_price'] = sales_month_df['res_cat_sales'].apply(lambda x: x[1])
sales_month_df['res_cat_mean_item_cnt_day'] = sales_month_df['res_cat_sales'].apply(lambda x: x[2])
sales_month_df['res_cat_std_item_price'] = sales_month_df['res_cat_sales'].apply(lambda x: x[3])
sales_month_df['res_cat_std_item_cnt_day'] = sales_month_df['res_cat_sales'].apply(lambda x: x[4])
sales_month_df['res_cat_max_item_price'] = sales_month_df['res_cat_sales'].apply(lambda x: x[3])
sales_month_df['res_cat_max_item_cnt_day'] = sales_month_df['res_cat_sales'].apply(lambda x: x[5])
sales_month_df['res_cat_min_item_price'] = sales_month_df['res_cat_sales'].apply(lambda x: x[6])
sales_month_df['res_cat_min_item_cnt_day'] = sales_month_df['res_cat_sales'].apply(lambda x: x[7])
sales_month_df['res_cat_skew_item_price'] = sales_month_df['res_cat_sales'].apply(lambda x: x[8])
sales_month_df['res_cat_skew_item_cnt_day'] = sales_month_df['res_cat_sales'].apply(lambda x: x[9])
sales_month_df.drop(labels=['res_cat_sales'], inplace=True, axis=1)

In [18]:
sales_month_df.drop(labels=['shop_id','item_id','item_category_id'], inplace=True, axis=1)

In [19]:
sales_month_df.head()

,index,date_block_num,item_price,item_cnt_day,ID_pair,res_cat_count,res_cat_mean_item_price,res_cat_mean_item_cnt_day,res_cat_std_item_price,res_cat_std_item_cnt_day,res_cat_max_item_price,res_cat_max_item_cnt_day,res_cat_min_item_price,res_cat_min_item_cnt_day,res_cat_skew_item_price,res_cat_skew_item_cnt_day
0,1549474,15,299.0,1.0,54-18394,4048,759.170188,1.358696,1337.134210,4.963847,1337.134210,22990.0,300.0,5.0,-1.0,9.849766
1,1549475,15,299.0,1.0,54-18394,4048,759.170188,1.358696,1337.134210,4.963847,1337.134210,22990.0,300.0,5.0,-1.0,9.849766
2,1549482,15,299.0,1.0,54-18394,4048,759.170188,1.358696,1337.134210,4.963847,1337.134210,22990.0,300.0,5.0,-1.0,9.849766
3,1562915,15,299.0,1.0,57-18394,3470,751.792670,1.251009,1703.931685,1.109768,1703.931685,23990.0,21.0,5.0,-1.0,9.666128
4,1564393,15,299.0,1.0,24-18394,1429,992.354084,1.172848,1806.535835,1.011941,1806.535835,22990.0,31.0,5.0,-1.0,8.068218


## EntitySet Processing (Recommended by FeatureTools)

In [20]:
es = ft.EntitySet(id="prediction_sales")

In [21]:
es = es.entity_from_dataframe(entity_id='sales',dataframe=sales_month_df, index='index')

In [22]:
es = es.normalize_entity(base_entity_id='sales',
                         new_entity_id='ids',
                         index='ID_pair',
                         additional_variables=["date_block_num",
                                               "res_cat_count",
                                               "res_cat_mean_item_price",
                                               "res_cat_mean_item_cnt_day",
                                               "res_cat_std_item_price",
                                               "res_cat_std_item_cnt_day",
                                               "res_cat_max_item_price",
                                               "res_cat_max_item_cnt_day",
                                               "res_cat_min_item_price",
                                               "res_cat_min_item_cnt_day",
                                               "res_cat_skew_item_price",
                                               "res_cat_skew_item_cnt_day"]
                        )

In [23]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity='ids')

In [28]:
#feature_matrix.info()

In [29]:
#feature_matrix.iloc[0]

# End of Case! 